<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Asi_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
Training a neural named entity recognition (NER) tagger 

In [12]:
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('version: {}, device: {}'.format(torch.__version__, device))

version: 1.5.0+cu101, device: cuda


In this assignment you are required to build a full training and testing pipeline for a neural sequentail tagger for named entities, using LSTM.

The dataset that you will be working on is called ReCoNLL 2003, which is a corrected version of the CoNLL 2003 dataset: https://www.clips.uantwerpen.be/conll2003/ner/

[Train data](https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing)

[Dev data](https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing)

[Test data](https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing)

As you can see, the annotated texts are labeled according to the IOB annotation scheme, for 3 entity types: Person, Organization, Location.

**Task 1:** Write a funtion for reading the data from a single file (of the ones that are provided above). The function recieves a filepath and then it encodes every sentence individually using a pair of lists, one list contains the words and one list contains the tags. Each list pair will be added to a general list (data), which will be returned back from the function.

In [4]:
import requests
import re

def read_data(filepath):
    data = []

    result = re.compile(".*drive.google.com/file/d/([^/]*)/.*").match(filepath)
    if result:
      filepath = 'https://docs.google.com/uc?export=download&id={}'.format(result.group(1))
    print(filepath)

    response = requests.get(filepath)
    words = []
    tags = []

    for line in response.text.split('\n'):
        if not line:
            if len(words) > 0:
                data.append((words, tags))
            words = []
            tags = []
        else:
            line = line.strip().split()
            words.append(line[0])
            tags.append(line[1])

    return data

train = read_data('https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing')
dev = read_data('https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing')
test = read_data('https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing')


https://docs.google.com/uc?export=download&id=1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz
https://docs.google.com/uc?export=download&id=1EAF-VygYowU1XknZhvzMi2CID65I127L
https://docs.google.com/uc?export=download&id=16gug5wWnf06JdcBXQbcICOZGZypgr4Iu


The following Vocab class can be served as a dictionary that maps words and tags into Ids. The UNK_TOKEN should be used for words that are not part of the training data.

In [5]:
UNK_TOKEN = 0

class Vocab:
    def __init__(self):
        self.word2id = {"__unk__": UNK_TOKEN}
        self.id2word = {UNK_TOKEN: "__unk__"}
        self.n_words = 1
        
        self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
        self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
        
    def index_words(self, words):
      word_indexes = [self.index_word(w) for w in words]
      return word_indexes

    def index_tags(self, tags):
      tag_indexes = [self.tag2id[t] for t in tags]
      return tag_indexes
    
    def index_word(self, w):
        if w not in self.word2id:
            self.word2id[w] = self.n_words
            self.id2word[self.n_words] = w
            self.n_words += 1
        return self.word2id[w]
            

**Task 2:** Write a function prepare_data that takes one of the [train, dev, test] and the Vocab instance, for converting each pair of (words,tags) to a pair of indexes. Each pair should be added to data_sequences, which will be returned back from the function.

In [6]:
vocab = Vocab()

def prepare_data(data, vocab):
    data_sequences = []
    for words, tags in data:
      iwords = vocab.index_words(words)
      itags = vocab.index_tags(tags)
      data_sequences.append((iwords, itags))

    return data_sequences, vocab

train_sequences, vocab = prepare_data(train, vocab)
dev_sequences, vocab = prepare_data(dev, vocab)
test_sequences, vocab = prepare_data(test, vocab)

**Task 3:** Write NERNet, a PyTorch Module for labeling words with NER tags. 

*input_size:* the size of the vocabulary

*embedding_size:* the size of the embeddings

*hidden_size:* the LSTM hidden size

*output_size:* the number tags we are predicting for

*n_layers:* the number of layers we want to use in LSTM

*directions:* could 1 or 2, indicating unidirectional or bidirectional LSTM, respectively

The input for your forward function should be a single sentence tensor.

In [7]:
class NERNet(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, bidirectional):
        super(NERNet, self).__init__()

        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.bidirectional = bidirectional

        in_features = self.hidden_size
        if bidirectional:
            in_features = self.hidden_size * 2

        self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size)
        self.lstm = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=n_layers, bidirectional=bidirectional, batch_first=True)
        self.out = nn.Linear(in_features=in_features, out_features=output_size)

    def forward(self, input_sentence):
        embedded = self.embedding(input_sentence)
        output, _ = self.lstm(embedded)
        output = output.contiguous()
        output = output.view(-1, output.shape[2])
        return self.out(output)


**Task 4:** write a training loop, which takes a model (instance of NERNet) and number of epochs to train on. The loss is always CrossEntropyLoss and the optimizer is always Adam.

In [14]:
import torch.optim as optim

def train_loop(model, n_epochs):
  # Loss function
  criterion = nn.CrossEntropyLoss()

  # Optimizer (ADAM is a fancy version of SGD)
  optimizer = optim.Adam(model.parameters(), lr=0.0001)
  
  print_every = 100

  for e in range(1, n_epochs + 1):

    loss = 0
    for counter, (sequence, labels) in enumerate(train_sequences):

        sequence_tensor = torch.LongTensor(sequence).to(device).view(1, -1)
        labels_tensor = torch.LongTensor(labels).to(device)

        # compute model output and loss
        output_batch = model(sequence_tensor)
        sentence_loss = criterion(output_batch, labels_tensor)

        optimizer.zero_grad()
        sentence_loss.backward()

        # updating weights
        optimizer.step()

        # averaging total loss
        loss += (sentence_loss.item() / len(sequence))

        if counter > 0 and counter % print_every == 0:
            loss = loss / print_every
            print('Epoch %d, %d/%d, Current Loss = %.4f' % (e, counter, len(train_sequences), loss))
            loss = 0



**Task 5:** write an evaluation loop on a trained model, using the dev and test datasets. This function print the true positive rate (TPR), also known as Recall and the opposite to false positive rate (FPR), also known as precision, of each label seperately (7 labels in total), and for all the 6 labels (except O) together. The caption argument for the function should be served for printing, so that when you print include it as a prefix.

In [9]:
def evaluate(model, caption):
  # TODO - your code goes here
  print(...)

**Task 6:** Train and evaluate a few models, all with embedding_size=300, and with the following hyper parameters (you may use that as captions for the models as well):

Model 1: (hidden_size: 500, n_layers: 1, directions: 1)

Model 2: (hidden_size: 500, n_layers: 2, directions: 1)

Model 3: (hidden_size: 500, n_layers: 3, directions: 1)

Model 4: (hidden_size: 500, n_layers: 1, directions: 2)

Model 5: (hidden_size: 500, n_layers: 2, directions: 2)

Model 6: (hidden_size: 500, n_layers: 3, directions: 2)

Model 4: (hidden_size: 800, n_layers: 1, directions: 2)

Model 5: (hidden_size: 800, n_layers: 2, directions: 2)

Model 6: (hidden_size: 800, n_layers: 3, directions: 2)

In [15]:
embedding_size = 100
hidden_size = 500
n_layers = 1
bidirectional = False

model = NERNet(vocab.n_words, hidden_size, hidden_size, len(vocab.tag2id), n_layers, bidirectional).to(device)

train_loop(model, 10)


Epoch 1, 100/1750, Current Loss = 0.1707
Epoch 1, 200/1750, Current Loss = 0.1780
Epoch 1, 300/1750, Current Loss = 0.1616
Epoch 1, 400/1750, Current Loss = 0.1214
Epoch 1, 500/1750, Current Loss = 0.1490
Epoch 1, 600/1750, Current Loss = 0.1248
Epoch 1, 700/1750, Current Loss = 0.1027
Epoch 1, 800/1750, Current Loss = 0.1481
Epoch 1, 900/1750, Current Loss = 0.1085
Epoch 1, 1000/1750, Current Loss = 0.0809
Epoch 1, 1100/1750, Current Loss = 0.1284
Epoch 1, 1200/1750, Current Loss = 0.0972
Epoch 1, 1300/1750, Current Loss = 0.0966
Epoch 1, 1400/1750, Current Loss = 0.1483
Epoch 1, 1500/1750, Current Loss = 0.0737
Epoch 1, 1600/1750, Current Loss = 0.0815
Epoch 1, 1700/1750, Current Loss = 0.0720
Epoch 2, 100/1750, Current Loss = 0.0598
Epoch 2, 200/1750, Current Loss = 0.0938
Epoch 2, 300/1750, Current Loss = 0.0802
Epoch 2, 400/1750, Current Loss = 0.0627
Epoch 2, 500/1750, Current Loss = 0.0766
Epoch 2, 600/1750, Current Loss = 0.0664
Epoch 2, 700/1750, Current Loss = 0.0465
Epoch 2,

**Task 6:** Download the GloVe embeddings from https://nlp.stanford.edu/projects/glove/ (use the 300-dim vectors from glove.6B.zip). Then intialize the nn.Embedding module in your NERNet with these embeddings, so that you can start your training with pre-trained vectors. Repeat Task 6 and print the results for each model.

Note: make sure that vectors are aligned with the IDs in your Vocab, in other words, make sure that for example the word with ID 0 is the first vector in the GloVe matrix of vectors that you initialize nn.Embedding with. For a dicussion on how to do that, check it this link:
https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222

In [ ]:
# TODO - your code goes here...

**Good luck!**